In [ ]:
import tdmclient.notebook
from Kalman import ekf
import time

In [2]:
await tdmclient.notebook.start()

In [ ]:
@tdmclient.notebook.sync_var 
def set_speed(right_speed,left_speed):
    global motor_right_target, motor_left_target
    motor_right_target=right_speed
    motor_left_target=left_speed

@tdmclient.notebook.sync_var
def stop_thymio():
    global motor_right_target,motor_left_target
    motor_right_target=0
    motor_left_target=0

@tdmclient.notebook.sync_var
def get_speed():
    global motor_right_speed, motor_left_speed
    return motor_right_speed,motor_left_speed


In [4]:
import matplotlib.pyplot as plt
import numpy as np


def speed_convesion(r_speed,l_speed):
    thymio_speed_to_mms = 0.430 # value found in covariance_estimation

    avg_thymio_speed = (r_speed + l_speed) / 2
    speed = avg_thymio_speed * thymio_speed_to_mms # [mm/s]
    return speed

def angular_vel_conversion(r_speed,l_speed):
    d = 95 # distance between the 2 wheels [mm]
    thymio_speed_to_mms = 0.430 # value found in covariance_estimation
    
    difference_speed = l_speed - r_speed
    omega = difference_speed * thymio_speed_to_mms / d # [rad/s]

    return omega

def get_velocity_vector(r_speed,l_speed):
    v=speed_convesion(r_speed,l_speed)
    w=angular_vel_conversion(r_speed,l_speed)
    return v,w


In [5]:
time_prev = time.time()


input_right_speed=100
input_left_speed=50

thymio_speed_to_mms = 0.430 #value found in covariance_estimation
L= 95 # distance between the 2 wheels in mm

obs_camera=[np.nan,np.nan,np.nan]
camera_state=False

set_speed(input_right_speed,input_left_speed)

input_v = speed_convesion(input_right_speed,input_left_speed)
input_w = angular_vel_conversion(input_right_speed,input_left_speed)
control_vect=[input_v,input_w]

x_est=[0,0,0,0,0]
P_est=np.diag([0.1,0.1,0.1,0.1,0.1])

while True:
    actual_time = time.time()
    Ts = actual_time - time_prev
    if Ts < 0.15:
        time.sleep(0.15-Ts)
    time_prev = actual_time

    v_right_odo, v_left_odo=get_speed()

    actual_v = speed_convesion(v_right_odo,v_left_odo)
    actual_w = angular_vel_conversion(v_right_odo,v_left_odo)
    obs_odometry=[actual_v,actual_w]

    x_est, P_est = ekf(x_est, control_vect, P_est, obs_camera, obs_odometry, camera_state,Ts)
    print(x_est)



[ 3.37689798e-02  0.00000000e+00 -2.81754029e-04  2.59778427e+01
 -2.70019602e-01]
[ 4.90686758e+00 -1.45183049e-03 -3.97895929e-02  2.47620619e+01
 -2.56315955e-01]
[ 4.95161552e+00 -3.97765556e-03 -4.01490801e-02  3.13237402e+01
 -2.23489310e-01]
[ 9.74290855 -0.19656131 -0.07402872 31.32042568 -0.22351414]
[ 9.77676749 -0.20117209 -0.07549371 31.05421058 -0.26269023]
[14.51484971 -0.56223683 -0.11186979 31.05411975 -0.26268338]
[14.64149951 -0.57280159 -0.11056025 33.97740173 -0.15638004]
[19.59643973 -1.11797199 -0.13996995 33.97740452 -0.15638032]
[19.6321057  -1.12307465 -0.1404192  32.3828773  -0.23470804]
[24.45887853 -1.8057801  -0.17513336 32.38287731 -0.23470804]
[24.43123828 -1.80640238 -0.17767241 28.79519001 -0.30464347]
[28.979319   -2.62920877 -0.21704819 28.79519    -0.30464347]
[29.0007525  -2.63501292 -0.21793753 31.85136808 -0.25149254]
[33.8129001  -3.70238233 -0.25445015 31.85136808 -0.25149254]
[33.84205054 -3.71024258 -0.25380771 31.18698154 -0.18715195]
[38.441

KeyboardInterrupt: 

In [6]:
stop_thymio()